In [ ]:
# there are two types of hacks on using autoregressive models as RM.
# one is to give a prompt and compare which one is better.
# (hey you can do this with robert, it has "multiple choice" capabilities. builtin!)
# (usually this is not good, since we will have lengthy answers with newline, breaking syntax)
# (unless you use special token as seen in ELI5 experiment: https://yjernite.github.io/lfqa.html)
# (replace special tokens (till nothing left) from training data first, then format the form.)
# the other is to export the hidden states, use fixed linear networks and get reward score.

# but reward modeling is multifold. we can use multiqa-bert, retribert, comparing prompts to answers.
# for sentence transformers, you can pass similarity score in dataset.

# train a multi-label model from autoregressive model using "AutoModelForSequenceClassification"
# (as hidden states to our linear neural network, as reward model)
# you can also use "SetFit" to train a level of labels based on existing sentence transformers
# (like truthfulness from 1 to 7, informative and so on.)

# from my point of view, using such large language model is not memory efficient
# compared to retrieval+generative based QA model (ILQA), which can also be fine-tuned.
# somebody using RETRO is saying the same thing.
# anyway, both approaches require a proper reward function/model or metric!

In [ ]:
# I think WebGPT, RETRO and LFQA have commondities. They all use references.

# WebGPT mimics actions taken during web browsing
# RETRO uses FAISS as backend
# LFQA combines retrieval BERT with generative BART

In [ ]:
# so let's begin investigation.
# give prompt and compare which is better?
# to predict logits, (decide is A or B) see: https://github.com/LouisCastricato/limited-data-scaling-laws/blob/79fd6fcb452d9b513d9520c144a447d5a11ccecd/critic_models.py#L55

special_token = "<P>" # hey you make sure this exists in the tokenizer, since it differs with model

def replaceTillNothingLeft(string:str, objective:str,target:str=""):
    while objective in string:
        string = string.replace(objective,target)
    return string

RTNL = lambda x: replaceTillNothingLeft(x,special_token) # shame we cannot typehint you! is it?
question = "What is the most beautiful thing in this world?"
ans_0 = "Dog."
ans_1 = "Cat."
mprompt = f"Given the question and two answers, find the better answer.{special_token}Question: {RTNL(question)}{special_token}A: {RTNL(ans_0)}{special_token}B: {RTNL(ans_1)}{special_token}Mark it as A or B."

# alright, we know the drill. leave it for now.
mprompt

In [ ]:
diag_0 =  lambda spec_token: f"question: {question}{spec_token}answer: {ans_0}"
diag_1 =  lambda spec_token: f"question: {question}{spec_token}answer: {ans_1}"

In [ ]:
# now export hidden states.
# we know "roberta" "gpt-like" and "FLAN-T5" (no example!)

# first, roberta. (seen in ILQL: https://github.com/Sea-Snell/Implicit-Language-Q-Learning/blob/0ee79b7cee11a56c20ecc54ad246c7033af0ee92/src/toxicity/reward_model.py)
# bert-like models seems have fixed hidden size?

# you want changes? it has special "attention mask" and pooling output.

def strip_from_end(str_item, strip_key):
    return strip_from_beginning(str_item[::-1], strip_key[::-1])[::-1]

def strip_from_beginning(str_item, strip_key):
    if str_item[:len(strip_key)] == strip_key:
        return str_item[len(strip_key):]
    return str_item

items = {}
spec_token = "</s>" # what is this token?
raw_strs = [diag_0(spec_token),diag_1(spec_token)] # what should be in this thing?


from transformers import RobertaModel, RobertaTokenizer

model_id = "deepset/roberta-base-squad2" # whatever.

model = RobertaModel.from_pretrained(model_id)
roberta_tokenizer = RobertaTokenizer.from_pretrained(model_id)

import torch

# use cpu instead?
device = "cpu"

# whatever. let's just input whatever we want.
# raw_strs = [strip_from_end(strip_from_beginning((raw_str[:raw_str.find('<|pad|>')] if raw_str.find('<|pad|>') != -1 else raw_str).strip(), '<a>'), '</a> </eod>').strip() for raw_str in raw_strs]
new_tokenized = roberta_tokenizer(raw_strs, padding=True)
items['tokens'] = torch.tensor(new_tokenized['input_ids']).to(device)
items['attn_mask'] = torch.tensor(new_tokenized['attention_mask']).to(device)
# items
roberta_tokenizer.sep_token # yeah we've got it.

In [ ]:
model.config.hidden_size # sure. but how do we decompose this model?

In [ ]:
# i do agree now, gpu is only for acceleration.
# if with enough RAM we can handle large model, for test purpose.
# now get the embedding
pooler_output = model(items['tokens'],attention_mask = items['attn_mask']).pooler_output
pooler_output, pooler_output.shape

# and now you can map this thing to reward. really?
# they are similar! damn.

In [ ]:
!pip install opendelta

In [1]:
# now, for gpt-like models, how to apply this?
# to show model structures, just print it
# seen in here: https://github.com/James4Ever0/limited-data-scaling-laws/blob/main/train_rm/train_rm.py


from transformers import AutoModelForCausalLM, AutoTokenizer
name = "EleutherAI/gpt-neo-125M"
model = AutoModelForCausalLM.from_pretrained(name)
# one crucial step: sending "transformer" in
transformer = model.transformer
tokenizer = AutoTokenizer.from_pretrained(name)

# if you want to use special tokens in gpt, you have to resize the model head.
# https://github.com/huggingface/transformers/issues/8039
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer.add_special_tokens({'sep_token': '[SEP]'})
# tokenizer.add_special_tokens({'pad_token': ' '}) # swapped. fuck!

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [2]:
# tokenizer.add_special_tokens({'sep_token': '[SEP]'}) # i guess this shit is to be blamed.
# since gpt-neo use only 50257 features. fuck.
# transformer

GPTNeoModel(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0.0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
       

In [3]:
# dir(tokenizer)
[tokenizer.sep_token_id,tokenizer.pad_token,tokenizer.pad_token_id] # i guess this is new to the model huh?
# no sep_token?
# it does have place.
# [(x,tokenizer.__dict__.get(x,None)) for x in dir(tokenizer)]
# can you add more?

Using pad_token, but it is not set yet.


[None, None, None]

In [ ]:
tokenizer.decode([0,50254,50255,50256])

In [ ]:
tokenizer.encode(" ") # space: 220, as replacement of padding?
tokenizer.encode(".")  # period: 13

In [ ]:
import torch
# model(torch.tensor([[50256]])) # starting from 50257 this shit cannot decode. what to do? just use attention mask.

In [4]:
# now let's encode
# spec_token = tokenizer.sep_token
# replace this token with others!
spec_token = ";" #no such token you fuck.
# spec_token = "[SEP]" # is it still avaliable?
# use it anyway.

question = "What is the most beautiful thing in this world?"
ans_0 = "Dog."
ans_1 = "Cat."

tokenizer.pad_token_id = 220
# tokenizer.pad_token = " "
# tokenizer.sep_token = ";"
# using reserved utf-8 codes? i don't think so.
# print("PAD TOKEN?",tokenizer.pad_token)
# print("SEP TOKEN?", tokenizer.sep_token)
# now it is using endoftext for padding. interesting?
# great?

# still, missing something.
# string = [f'Query: {question}{spec_token}Answer: {answer}' for answer in [ans_0, ans_1]]
string = [f'Query: {question}{spec_token}Answer: {answer}' for answer in [ans_0, ans_1,"Frog and Cat."]]
print("STRING?",string)
# you must use padding. fuck.
out = tokenizer(string, return_tensors="pt", padding=True, truncation=False) # use padding here?
out # do not use endoftext

STRING? ['Query: What is the most beautiful thing in this world?;Answer: Dog.', 'Query: What is the most beautiful thing in this world?;Answer: Cat.', 'Query: What is the most beautiful thing in this world?;Answer: Frog and Cat.']


{'input_ids': tensor([[20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25,  8532,    13,   220,   220],
        [20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25,  5181,    13,   220,   220],
        [20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25, 28328,   290,  5181,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
# check fidality.
tokenizer.decode([20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25,  8532,    13, 50256, 50256])

'Query: What is the most beautiful thing in this world?;Answer: Dog.<|endoftext|><|endoftext|>'

In [ ]:
# what is token 13?
tokenizer.decode([13]) # period. fuck.

In [10]:
hidden_size = model.config.hidden_size
hidden_size

768

In [11]:
from torch import nn
v_head = nn.Linear(hidden_size, 1, bias=False)
# model
# inspect model structure?

In [6]:
# let's get hidden states.
input_ids = out['input_ids']
attention_mask = out['attention_mask']

print("INPUT?",input_ids)
print("trying")

# print(dir(model))
output = transformer( 
            input_ids,
            attention_mask=attention_mask,
#     output_hidden_states=True
        )

# out of range?
# transformer_outputs
# no hidden states this time.
# print(len(transformer_outputs)) #2
hidden_states = output[0]
hidden_states.shape
# print("output_1?",len(transformer_outputs[1]))
# hidden_states.shape # strange. # why the fuck?
# print(dir(transformer_outputs)) # what the fuck is going on?
# print(len(transformer_outputs.hidden_states)) # None? 13 hidden states? wtf?
# #size: 17,768 (original)
# torch.Size([3, 19, 768]) fucking A!
# torch.Size([3, 19, 50257])
# calculate average score?
# input_ids, input_ids.shape # 19 now?

# attention_mask, attention_mask.shape
# this attention mask, is suitable for directing attention to non "pad" or "section" tokens

INPUT? tensor([[20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25,  8532,    13,   220,   220],
        [20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25,  5181,    13,   220,   220],
        [20746,    25,  1867,   318,   262,   749,  4950,  1517,   287,   428,
           995,    30,    26, 33706,    25, 28328,   290,  5181,    13]])
trying


torch.Size([3, 19, 768])

In [7]:
model.config.hidden_size

768

In [13]:
r = v_head(hidden_states)
r.shape

torch.Size([3, 19, 1])

In [14]:
rwList = r.squeeze(-1)
rwList.shape

torch.Size([3, 19])

In [16]:
import torch
rw_mean = torch.mean(rwList)
rw_mean
# great. what now?

tensor(0.0589, grad_fn=<MeanBackward0>)

In [ ]:
# saying you should install "apex" if you want accelerated T5
# from document?
# apex is provided by nvidia: https://github.com/NVIDIA/apex

# installation:

# git clone https://github.com/NVIDIA/apex
# cd apex
# pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

# or:

# pip install git+https://github.com/NVIDIA/apex.git

In [17]:
# you tune FLAN-T5 for this job?
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

In [26]:
# dir(tokenizer)
# tokenizer.all_special_tokens
tokenizer.eos_token, tokenizer.pad_token

('</s>', '<pad>')

In [54]:
# dir(model.config)
# nothing here!
token_ids = (model.config.sep_token_id,model.config.pad_token_id) # it does have 
print(token_ids)
minput_0 = "Question: What is the most beautiful animal in this world? A: Dog. B: Cat. Answer either A or B:"
minput_1 = "Question: What is the most beautiful animal in this world? A: Dog. B: Frog. Answer either A or B:"
minputs = [minput_0,minput_1] # no padding? fuck?
memb_input = tokenizer(minputs,return_tensors="pt",padding=True)
print("INPUT VECTOR?",memb_input)
# output = model(input_ids = memb_input['input_ids'])
decoder_input_ids = torch.ones((memb_input['input_ids'].shape[0], 1), dtype=torch.long) * model.config.decoder_start_token_id
output = model(input_ids = memb_input['input_ids'],attention_mask=memb_input['attention_mask'],decoder_input_ids=decoder_input_ids)
# print("OUTPUT?",output)
# great. seen in openai.
hidden_params = output[2]
hidden_params

(None, 0)
INPUT VECTOR? {'input_ids': tensor([[11860,    10,   363,    19,     8,   167,   786,  2586,    16,    48,
           296,    58,    71,    10,  6751,     5,   272,    10,  3431,     5,
         11801,   893,    71,    42,   272,    10,     1,     0],
        [11860,    10,   363,    19,     8,   167,   786,  2586,    16,    48,
           296,    58,    71,    10,  6751,     5,   272,    10,   377,  3822,
             5, 11801,   893,    71,    42,   272,    10,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}


tensor([[[ 1.4192e-02,  2.1341e-01, -2.0558e-01,  ..., -3.9713e-02,
          -6.6147e-02, -4.7994e-02],
         [ 6.8610e-03,  7.3397e-02, -4.6221e-02,  ..., -7.0633e-02,
          -9.5900e-03, -9.7069e-02],
         [ 9.3572e-02,  2.8400e-02, -5.0034e-02,  ...,  6.7745e-02,
           5.5376e-02,  8.1821e-02],
         ...,
         [ 3.3754e-02, -4.9938e-03, -2.8832e-02,  ...,  5.8913e-02,
           2.2033e-01,  1.1926e-01],
         [ 9.4769e-03,  6.4210e-03,  1.4606e-02,  ...,  1.4873e-03,
          -2.6005e-03,  8.3175e-04],
         [-2.8552e-01,  1.2683e-01,  6.7264e-02,  ...,  2.2612e-01,
           7.2569e-02,  1.8119e-01]],

        [[ 9.6625e-03,  1.9815e-01, -1.9542e-01,  ..., -4.1598e-02,
          -6.1196e-02, -4.0653e-02],
         [ 1.7699e-04,  5.5468e-02, -2.8853e-02,  ..., -8.6591e-02,
          -1.3577e-02, -8.6231e-02],
         [ 7.5097e-02,  1.6171e-02, -5.4481e-02,  ...,  5.4896e-02,
           4.7474e-02,  9.7275e-02],
         ...,
         [ 1.6699e-01, -6

In [55]:
val = v_head(hidden_params).squeeze(-1)
print("VAL_SPARSE:",val.shape)
val_sum = torch.sum(val)
# you can use some non-linear magic.
print(val_sum)

VAL_SPARSE: torch.Size([2, 28])
tensor(-2.2909, grad_fn=<SumBackward0>)


In [57]:
# let's load sentence transformers.
# find similar query/answer pairs?
# if you just want to use this "retrival" based model without training, you are doomed.
# retrival based model is for building train sets. is it?
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 916.4 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=001ee7ea8059834f57882fca84b0514437fbc33baf6099c8e356d9f777fb0cbe
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [60]:
# model_id = "yjernite/retribert-base-uncased" # working or not?
model_id = "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1" # oh shit. train it first?
from sentence_transformers import CrossEncoder
st = CrossEncoder(model_id,num_labels=1) # for training?
# st = SentenceTransformer(model_id)
st

Downloading:   0%|          | 0.00/891 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/435 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [63]:
# now let's compare.
query = "What is the most beautiful animal in this world?"
# ans_0 = "Samoyed." # better than dog!
ans_0 = "Dog."
ans_1 = "Cat."
mdata = [[query, ans] for ans in [ans_0, ans_1]]
scores = st.predict(mdata)
scores # so it prefers cat.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([0.19621429, 0.68443924], dtype=float32)

In [75]:
# let's train this shit to prefer do      g.
# i think bi-encoders (retrieval) based are more memory efficient? or not?
from sentence_transformers import InputExample # you can use discrete grades.
# this is asymmetric. query and answer.
train_samples = [
  InputExample(texts=[query, ans_0], label=0.8), # ugly cat!
  InputExample(texts=[query, ans_1], label=0.1),
]
train_batch_size = 1 # you can raise this thing.
num_epochs = 20
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator

import math
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)
warmup_steps = 0

# use the same sample?
# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator.from_input_examples(train_samples, name='sts-dev')

# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader

model_save_path = "./model_qa"

# Train the model
# wtf? you cannot fit this shit?
# the name is incorrect.
st.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

print("TRAIN COMPLETE")

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

TRAIN COMPLETE


In [1]:
scores = st.predict(mdata)# how can you have >1 things?
scores # fuck. 
# array([ 1.9676722, -2.1964266], dtype=float32)
# this is how you train shits? anyway, it does learn some human preference.

NameError: name 'st' is not defined

In [1]:
# load another model, be it multilingual QA
# oops nothing found on sentence bert.
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
# loading your own model?
# note: do not fine-tune that multi-language sentence similarity model since that will break shit.
model_id = "sentence-transformers/all-MiniLM-L6-v2" # just load english shit.
# guide on how to make models multilingual:
# https://www.sbert.net/examples/training/multilingual/README.html
st = SentenceTransformer(model_id)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 613.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ec720d91a9313d959d95c14746314754019b3eae3396cb04c330cbf56191f482
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [2]:
# st
from sentence_transformers import util


def testsim():
    # now let's compare.
    query = "What is the most beautiful animal in this world?"
    # ans_0 = "Samoyed." # better than dog!
    ans_0 = "Dog."
    ans_1 = "Cat."
    query_embedding = st.encode(query)

    passage_embedding = st.encode([ans_0, ans_1])

    print("Similarity:", util.cos_sim(query_embedding, passage_embedding))# shit?

testsim()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity: tensor([[0.3492, 0.3051]])


In [3]:
# again, how to train?

from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader               

train_examples = [InputExample(texts=[query, ans_0], label=0.8),
   InputExample(texts=[query, ans_1], label=0.3)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
st.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
print("TRAINING COMPLETE.")
# then you must test the result.

NameError: name 'query' is not defined

In [ ]:
testsim()

In [1]:
# use this.
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# i think it should not be too different. let's make it binary.

model_id = "EleutherAI/gpt-neo-125M"

num_labels = 1  # etc.

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id,
    num_labels=num_labels,
    problem_type="multi_label_classification",  # this is important
                                                          )
print("model loaded.")

# you should train this model.

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['transformer.h.3.attn.attention.bias', 'score.weight', 'transformer.h.9.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.5.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded.


In [2]:
# tokenizer.add_special_tokens({"pad_token":"[PAD]"})

# tutorial: 
# handle the damn missing special token problem for causal LM:
# https://github.com/huggingface/transformers/issues/2630

# this is bullshit. but it is needed! we just need to remap and mask it
# tokenizer.pad_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# in order to get a proper view among all "affected" tokens after change:
tokenizer.pad_token, tokenizer.pad_token_id, tokenizer.eos_token, tokenizer.eos_token_id
# the same? we are good.

('<|endoftext|>', 50256, '<|endoftext|>', 50256)

In [3]:
# dir(model.config)
# None? Let's set it.
model.config.pad_token_id=tokenizer.pad_token_id # useless fuck.
model.config.pad_token_id

50256

In [4]:
tokenizer.eos_token

'<|endoftext|>'

In [5]:
# now encode this.
data = ["sentence a: aaa", "sentence b: bb super b"] # wrong?

tokenizer.padding_side="left" # important!

input_data = tokenizer(data,padding=True,return_tensors="pt") # set padding to left. mask these fucks.
input_data

{'input_ids': tensor([[50256, 50256, 34086,   594,   257,    25,   257,  7252],
        [34086,   594,   275,    25,   275,    65,  2208,   275]]), 'attention_mask': tensor([[0, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [10]:
# let's input.
# https://www.alexanderjunge.net/blog/til-multi-label-automodelforsequenceclassification/

output = model(input_data['input_ids'],
      # yeah you of course see the damn difference.
      attention_mask = input_data['attention_mask'] # and you must be fucking kidding me.
      # transformers/src/transformers/models/t5/modeling_t5.py
      # https://cocomanga.xyz/comic/45241/650372.html#p=5
     )[0] # yeah you've got it, but where's the fucking mask?
# ,attn_mask=input_data['attention_mask']) # still not fucking working. not allowing padding shit.

# digits are not good. how to calculate loss and backpropagate?
# just omit?
output, output.dtype

(tensor([[-40.5820],
         [-39.3169]], grad_fn=<IndexBackward0>),
 torch.float32)

In [18]:
# to make it trainable?

import torch

# the bot prefers "unsqueeze"
# labels = torch.tensor([[0],[1]]) # shall be float32.
# labels = torch.tensor([float(0),float(1)]).unsqueeze(0) # 0? are you fucking sure? shape mismatch?
labels = torch.tensor([float(0),float(1)]).unsqueeze(-1) # 0? are you fucking sure? shape mismatch?
labels, labels.dtype # match? but wtf?

# note: nvidia/apex is not compatible below Turing architecture.

(tensor([[0.],
         [1.]]),
 torch.float32)

In [31]:
# now let's deal with the loss.
# to make it trainable?

import torch

# the bot prefers "unsqueeze"
# labels = torch.tensor([[0],[1]]) # shall be float32.
# labels = torch.tensor([float(0),float(1)]).unsqueeze(0) # 0? are you fucking sure? shape mismatch?
labels = torch.tensor([float(0),float(1)]).unsqueeze(-1) # 0? are you fucking sure? shape mismatch?
labels, labels.dtype # match? but wtf?

# from torch.nn import CrossEntropyLoss # loss not good?
# from torch.nn import MSELoss
from torch.nn import L1Loss
# from torch.nn import L2Loss
# from torch.nn import NLLLoss
from transformers import AdamW

# optimizer = AdamW(model.parameters(), lr=1) # too high? just for demo.
# this is bullshit.
# optimizer = AdamW(model.parameters(), lr=0.02) # too high? just for demo.
optimizer = AdamW(model.parameters(), lr=2e-5) # too low?
# loss_fn = MSELoss()
loss_fn = L1Loss()
# loss_fn = CrossEntropyLoss()
# loss_fn = NLLLoss()


# alter output?

output = model(input_data['input_ids'],
      # yeah you of course see the damn difference.
      attention_mask = input_data['attention_mask'] # and you must be fucking kidding me.
      # transformers/src/transformers/models/t5/modeling_t5.py
      # https://cocomanga.xyz/comic/45241/650372.html#p=5
)[0]

labels = torch.tensor([float(0),float(1)])


for _ in range(10000): # simulate training
    output = output.reshape(-1)
#     labels = labels.unsqueeze(0)

    # it selects moutput[0] as output.
    print("OUTPUT?",output,output.dtype)
    print("LABELS?",labels,labels.dtype)
    loss = loss_fn(output, labels) # no loss? wtf?
    print('LOSS?',loss)
    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    labels = torch.tensor([float(0),float(1)])
    output = model(input_data['input_ids'],
      # yeah you of course see the damn difference.
      attention_mask = input_data['attention_mask'] # and you must be fucking kidding me.
      # transformers/src/transformers/models/t5/modeling_t5.py
      # https://cocomanga.xyz/comic/45241/650372.html#p=5
    )[0]
    

print("MODEL TRAINED.")

OUTPUT? tensor([  96.7552, -101.6096], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS? tensor(99.6824, grad_fn=<L1LossBackward0>)
OUTPUT? tensor([  96.8062, -101.2893], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS? tensor(99.5477, grad_fn=<L1LossBackward0>)
OUTPUT? tensor([  96.8513, -101.0086], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS? tensor(99.4300, grad_fn=<L1LossBackward0>)
OUTPUT? tensor([  96.8945, -100.7339], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS? tensor(99.3142, grad_fn=<L1LossBackward0>)
OUTPUT? tensor([  96.9399, -100.4571], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS? tensor(99.1985, grad_fn=<L1LossBackward0>)
OUTPUT? tensor([  96.9869, -100.1779], grad_fn=<ReshapeAliasBackward0>) torch.float32
LABELS? tensor([0., 1.]) torch.float32
LOSS?

KeyboardInterrupt: 

In [32]:
model(input_data['input_ids'],
      attention_mask = input_data['attention_mask']
     )[0] # test if we are really trained. not working?

tensor([[ 99.8982],
        [-76.3190]], grad_fn=<IndexBackward0>)